In [107]:
import pandas as pd
import numpy as np
import math
import datetime as dt
import datetime
import warnings

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
from itertools import cycle

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode

from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.initializers import Orthogonal
from keras.layers import Activation

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional, BatchNormalization
import tensorflow as tf

from keras_tuner.tuners import RandomSearch

import openpyxl
import os
import requests
import csv
import time
import json
import pytz

from transformers import pipeline

init_notebook_mode(connected=True)
warnings.filterwarnings('ignore')

# Importing and cleaning the data:

In [108]:
# Data cutoffs:

initial_cutoff = '2015-01-01' # Needed to compute sharpe ratio for 2016
halving_date = '2016-07-09'# Training start date
test_start_date = '2020-09-30' # Test start date
cutoff_date = '2022-09-30' # Test end date

In [109]:
# Function for formatting JSON files:

def format_json(data, column):
    
    # Convert the list of dictionaries to a DataFrame
    data = pd.DataFrame(data, columns=[column])

    # Split the specified column into 'time' and 'active_addresses' columns
    data[['time', column]] = pd.DataFrame(data[column].tolist(), index=data.index)

    # Convert the 'time' column to datetime format
    data['time'] = pd.DatetimeIndex([pd.Timestamp(x) for x in data['time']])

    # Convert the date range to Timestamp objects with UTC timezone
    initial_cutoff = pd.to_datetime('2015-01-01', utc=True)
    cutoff_date = pd.to_datetime('2022-10-10', utc=True)

    # Filter the DataFrame based on the given date range

    data = data[(data['time'] >= initial_cutoff) & (data['time'] <= cutoff_date)]

    data['time'] = data['time'].dt.date

    data.sort_values(by='time', ascending=False).head(15)

    return data


In [110]:
# BTC

df_btc = pd.read_csv('raw_data/btc-usd-coingecko.csv')

df_btc.rename(columns={'price': 'btc_close'}, inplace=True)
df_btc.rename(columns={'snapped_at': 'time'}, inplace=True)
df_btc.rename(columns={'total_volume': 'btc_trading_volume'}, inplace=True)

df_btc['time'] = pd.to_datetime(df_btc['time']).dt.strftime('%Y-%m-%d')


df_btc['btcusd_dreturn'] = df_btc['btc_close'].pct_change()

# Tomorrows return (will serve as our y-values)
df_btc['btc_tmrw_return'] = df_btc['btcusd_dreturn'].shift(-1)

# Define the time period for RSI calculation (typically set to 14)
rsi_period = 14

# Calculate the daily price change for each asset
df_btc['price_change'] = df_btc['btc_close'].diff()

# Separate the price changes into up periods and down periods
df_btc['up_period'] = np.where(df_btc['price_change'] > 0, df_btc['price_change'], 0)
df_btc['down_period'] = np.where(df_btc['price_change'] < 0, abs(df_btc['price_change']), 0)

# Calculate the average gain and average loss over the selected period of time
df_btc['avg_gain'] = df_btc['up_period'].rolling(window=rsi_period).mean()
df_btc['avg_loss'] = df_btc['down_period'].rolling(window=rsi_period).mean()

# Calculate the relative strength (RS) as the ratio of the average gain to the average loss
df_btc['rs'] = df_btc['avg_gain'] / df_btc['avg_loss']

# Calculate the RSI using the formula RSI = 100 - (100 / (1 + RS))
df_btc['btc_rsi'] = 100 - (100 / (1 + df_btc['rs']))

df_btc = df_btc.drop(columns=['price_change','up_period', 'down_period', 'avg_gain', 'avg_loss', 'rs', 'market_cap'])

In [111]:
# ETH

df_eth = pd.read_csv('raw_data/eth-usd-coingecko.csv')

df_eth.rename(columns={'price': 'eth_close'}, inplace=True)
df_eth.rename(columns={'snapped_at': 'time'}, inplace=True)
df_eth.rename(columns={'total_volume': 'eth_trading_volume'}, inplace=True)

df_eth['time'] = pd.to_datetime(df_eth['time']).dt.strftime('%Y-%m-%d')

df_eth['ethusd_dreturn'] = df_eth['eth_close'].pct_change()

# Tomorrows return (will serve as our y-values)
df_eth['eth_tmrw_return'] = df_eth['ethusd_dreturn'].shift(-1)

# Define the time period for RSI calculation (typically set to 14)
rsi_period = 14

# Calculate the daily price change for each asset
df_eth['price_change'] = df_eth['eth_close'].diff()

# Separate the price changes into up periods and down periods
df_eth['up_period'] = np.where(df_eth['price_change'] > 0, df_eth['price_change'], 0)
df_eth['down_period'] = np.where(df_eth['price_change'] < 0, abs(df_eth['price_change']), 0)

# Calculate the average gain and average loss over the selected period of time
df_eth['avg_gain'] = df_eth['up_period'].rolling(window=rsi_period).mean()
df_eth['avg_loss'] = df_eth['down_period'].rolling(window=rsi_period).mean()

# Calculate the relative strength (RS) as the ratio of the average gain to the average loss
df_eth['rs'] = df_eth['avg_gain'] / df_eth['avg_loss']

# Calculate the RSI using the formula RSI = 100 - (100 / (1 + RS))
df_eth['eth_rsi'] = 100 - (100 / (1 + df_eth['rs']))

df_eth = df_eth.drop(columns=['price_change','up_period', 'down_period', 'avg_gain', 'avg_loss', 'rs', 'market_cap'])

In [112]:
# ETH/BTC

# Merge the two data frames on their timestamp columns
btceth = pd.merge(df_eth, df_btc, on='time')

# Calculate the ETH/BTC closing price by dividing the ETH closing price by the BTC closing price
btceth['ethbtc_close'] = btceth['eth_close'] / btceth['btc_close']

btceth['ethbtc_dreturn'] = btceth['ethbtc_close'].pct_change()

# Price lags
btceth['eth_lag1'] = btceth['eth_close'].shift(1)
btceth['eth_lag2'] = btceth['eth_close'].shift(2)
btceth['eth_lag3'] = btceth['eth_close'].shift(3)
btceth['eth_lag4'] = btceth['eth_close'].shift(4)
btceth['eth_lag5'] = btceth['eth_close'].shift(5)
btceth['eth_lag6'] = btceth['eth_close'].shift(6)
btceth['eth_lag7'] = btceth['eth_close'].shift(7)
btceth['eth_lag8'] = btceth['eth_close'].shift(8)
btceth['eth_lag9'] = btceth['eth_close'].shift(9)
btceth['eth_lag10'] = btceth['eth_close'].shift(10)

# Price lags
btceth['btc_lag1'] = btceth['btc_close'].shift(1)
btceth['btc_lag2'] = btceth['btc_close'].shift(2)
btceth['btc_lag3'] = btceth['btc_close'].shift(3)
btceth['btc_lag4'] = btceth['btc_close'].shift(4)
btceth['btc_lag5'] = btceth['btc_close'].shift(5)
btceth['btc_lag6'] = btceth['btc_close'].shift(6)
btceth['btc_lag7'] = btceth['btc_close'].shift(7)
btceth['btc_lag8'] = btceth['btc_close'].shift(8)
btceth['btc_lag9'] = btceth['btc_close'].shift(9)
btceth['btc_lag10'] = btceth['btc_close'].shift(10)

btceth['time'] = pd.to_datetime(btceth['time'])

In [113]:
# 3-month US treasury (risk-free rate)

ust3 = pd.read_csv('raw_data/us-treasury-3-month.csv')

ust3.rename(columns = {'DATE':'time'}, inplace=True)
ust3.rename(columns = {'TB3MS':'3m_treasury'}, inplace=True)

ust3 = ust3[(ust3['time'] >= '2015-01-01') & (ust3['time'] <= '2022-11-01')]

ust3['time'] = pd.to_datetime(ust3['time'])

# Create a new DataFrame with daily data by reindexing the original DataFrame
# to include all dates between the first and last dates in the 'time' column
first_date = ust3['time'].min()
last_date = ust3['time'].max()

idx = pd.date_range(start=first_date, end=last_date, freq='D')
ust3_daily = ust3.set_index('time').reindex(idx).rename_axis('time').reset_index()

# Forward fill the missing values in the 'TB3MS' column
ust3_daily['3m_treasury'] = ust3_daily['3m_treasury'].ffill()

ust3_daily = ust3_daily[(ust3_daily['time'] >= initial_cutoff) & (ust3_daily['time'] <= cutoff_date)]

In [114]:
# Merging price data

price_data = pd.merge(btceth, ust3_daily, on='time')

In [115]:
# Total market capitalization BTC (USD denominated) 
btc_marketcap = pd.read_json('raw_data/blockchain-data/bitcoin-market-cap-usd.json')

btc_marketcap = format_json(btc_marketcap, 'Market Cap')
btc_marketcap.rename(columns={'Market Cap': 'btc_mc'}, inplace=True)

# Total market capitalization ETH (USD denominated) 
eth_marketcap = pd.read_json('raw_data/ethereum-market-cap-usd.json')

eth_marketcap = format_json(eth_marketcap, 'Market Cap')
eth_marketcap.rename(columns={'Market Cap': 'eth_mc'}, inplace=True)

In [116]:
# Blockhain data

# BTC active wallet addresses
btc_active = pd.read_json('raw_data/bitcoin-number-of-active-addresses.json')

btc_active = format_json(btc_active,'Active Addresses')

btc_active.rename(columns={'Active Addresses': 'btc_active'}, inplace=True)

btc_active.sort_values(by='time', ascending=True).head(10)

# ETH active wallet addresses
eth_active = pd.read_json('raw_data/ethereum-addresses.json')

eth_active = format_json(eth_active,'Active Addresses')

eth_active.rename(columns={'Active Addresses': 'eth_active'}, inplace=True)

eth_active.sort_values(by='time', ascending=True).head(15)

active_merged = pd.merge(eth_active, btc_active, on='time')

# Transaction volume on-chain BTC (USD denominated) 
btc_volume = pd.read_json('raw_data/bitcoin-total-transfer-volume-usd.json')

btc_volume = format_json(btc_volume, 'Transfer Volume (Total)')
btc_volume.rename(columns={'Transfer Volume (Total)': 'btc_chainvolume'}, inplace=True)


# Transaction volume on-chain ETH (USD denominated) 
eth_volume = pd.read_json('raw_data/ethereum-total-transfer-volume-usd.json')

eth_volume = format_json(eth_volume, 'Transfer Volume (Total)')
eth_volume.rename(columns={'Transfer Volume (Total)': 'eth_chainvolume'}, inplace=True)


# Merging and calculating Network Value to Transactions Ratio (NVT):

# BTC
btc_data = pd.merge(btc_marketcap, btc_volume, on='time')

btc_data['btc_nvt'] = btc_data['btc_mc'] / btc_data['btc_chainvolume']

# ETH
eth_data = pd.merge(eth_marketcap, eth_volume, on='time')

eth_data['eth_nvt'] = eth_data['eth_mc'] / eth_data['eth_chainvolume']

# Exchange balances of ETH and BTC (In USD)

#BTC
btc_exflow = pd.read_json("raw_data/bitcoin-net-transfer-volume-from-to-exchanges-usd-all-exchanges.json")
btc_exflow = format_json(btc_exflow, 'Exchange Netflow Volume')
btc_exflow.rename(columns={'Exchange Netflow Volume': 'btc_exflow'}, inplace=True)

# ETH
eth_exflow = pd.read_json('raw_data/ethereum-net-transfer-volume-from-to-exchanges-usd-all-exchanges.json')
eth_exflow = format_json(eth_exflow, 'Exchange Netflow Volume')
eth_exflow.rename(columns={'Exchange Netflow Volume': 'eth_exflow'}, inplace=True)



In [135]:
# Creating the ultimate data set

df_merged2 = pd.merge(active_merged, eth_data, on='time')
df_merged3 = pd.merge(df_merged2, btc_data, on='time')
df_merged4 = pd.merge(df_merged3, btc_exflow, on='time')
df_merged5 = pd.merge(df_merged4, eth_exflow, on='time')

df_merged5['time'] = pd.to_datetime(df_merged5['time'])

df_final = pd.merge(df_merged5, price_data, on='time')

## Financial metrics

# Calculating daily risk free return:
df_final['ust3_dreturn'] = (1 + (3 - df_final['3m_treasury'] / 100) / 365) ** (1/365) - 1

# Sample portfolio for comparison

df_final['btc_alloc%'] = 0.3
df_final['eth_alloc%'] = 0.3
df_final['ust3_alloc%'] = 0.4

df_final['btc_alloc_usd'] = 300000
df_final['eth_alloc_usd'] = 300000
df_final['ust3_alloc_usd'] = 400000

## Sharpe ratio:

# Calculate example portfolio simple returns
df_final['portfolio_return'] = df_final['btc_alloc%'] * df_final['btcusd_dreturn'] \
                             + df_final['eth_alloc%'] * df_final['ethusd_dreturn'] \
                             + df_final['ust3_alloc%'] * df_final['ust3_dreturn']

# Calculate example daily portfolio volatility 
df_final['portfolio_volatility'] = df_final['portfolio_return'].rolling(window=252).std() * np.sqrt(252)

# Calculate example daily Sharpe ratio
df_final['daily_sharpe'] = (df_final['portfolio_return'] - df_final['ust3_dreturn']) / df_final['portfolio_volatility']

df_final = df_final[(df_final['time'] >= halving_date) & (df_final['time'] <= cutoff_date)]

df_final.set_index('time', inplace=True)

In [136]:
# Creating the dataset our deep learning model will work with:

df = df_final.drop(columns=['3m_treasury', 'ust3_dreturn', 'btc_alloc%', 'eth_alloc%', 'ust3_alloc%', 'btc_alloc_usd', 'eth_alloc_usd', 'ust3_alloc_usd', 'portfolio_return', 'portfolio_volatility', 'daily_sharpe'])

In [138]:
df.to_csv('df.csv', index=False)

In [132]:
# Creating the dataset our deep learning model will work with:

df = df_final.drop(columns=['3m_treasury', 'ust3_dreturn', 'btc_alloc%', 'eth_alloc%', 'ust3_alloc%', 'btc_alloc_usd', 'eth_alloc_usd', 'ust3_alloc_usd', 'portfolio_return', 'portfolio_volatility', 'daily_sharpe'])

df.reset_index(inplace=True)

df.to_csv('df.csv', index=False)

df.set_index('time', inplace=True)

# Printing all our dependent and independent variables
print(df.columns)

Index(['eth_active', 'btc_active', 'eth_mc', 'eth_chainvolume', 'eth_nvt',
       'btc_mc', 'btc_chainvolume', 'btc_nvt', 'btc_exflow', 'eth_exflow',
       'eth_close', 'eth_trading_volume', 'ethusd_dreturn', 'eth_tmrw_return',
       'eth_rsi', 'btc_close', 'btc_trading_volume', 'btcusd_dreturn',
       'btc_tmrw_return', 'btc_rsi', 'ethbtc_close', 'ethbtc_dreturn',
       'eth_lag1', 'eth_lag2', 'eth_lag3', 'eth_lag4', 'eth_lag5', 'eth_lag6',
       'eth_lag7', 'eth_lag8', 'eth_lag9', 'eth_lag10', 'btc_lag1', 'btc_lag2',
       'btc_lag3', 'btc_lag4', 'btc_lag5', 'btc_lag6', 'btc_lag7', 'btc_lag8',
       'btc_lag9', 'btc_lag10'],
      dtype='object')


In [133]:
df_portfolio = df_final[['3m_treasury', 'ust3_dreturn', 'btc_alloc%', 'eth_alloc%', 'ust3_alloc%', 'btc_alloc_usd', 'eth_alloc_usd', 'ust3_alloc_usd', 'portfolio_return', 'portfolio_volatility', 'daily_sharpe']]

In [134]:
# Only run once to avoid issues
df_portfolio.reset_index(inplace=True)

prices = pd.merge(df_eth, df_btc, on = 'time')
prices = prices[['time', 'btcusd_dreturn', 'ethusd_dreturn']]

df_portfolio['time'] = pd.to_datetime(df_portfolio['time']).dt.strftime('%Y-%m-%d')
df_portfolio = df_portfolio[(df_portfolio['time'] >= test_start_date) & (df_portfolio['time'] <= cutoff_date)]

# Exporting to csv
df_portfolio.to_csv('df_portfolio.csv', index=False)

In [121]:
# 60% crypto and 40% 3-month US treasury portfolio

df_hodl = pd.merge(df_portfolio, prices, on='time')
# Setting up the initial asset allocations
df_hodl['btc_alloc_usd'] = 300000
df_hodl['eth_alloc_usd'] = 300000
df_hodl['ust3_alloc_usd'] = 400000
# Initial allocation %
df_hodl['btc_alloc%'] = 0.3
df_hodl['eth_alloc%'] = 0.3
df_hodl['ust3_alloc%'] = 0.4

df_hodl['portfolio_balance_60_40'] = df_hodl['btc_alloc_usd'] + df_hodl['eth_alloc_usd'] + df_hodl['ust3_alloc_usd']

for index, row in df_hodl.iterrows():
    if index == df_hodl.index[-1]:
        break

    next_row = df_hodl.loc[index + 1]

    temp_btc_alloc_usd = df_hodl.at[index, 'btc_alloc_usd']
    temp_eth_alloc_usd = df_hodl.at[index, 'eth_alloc_usd']
    temp_ust3_alloc_usd = df_hodl.at[index, 'ust3_alloc_usd']   
    
    # Calculating allocation % before profit/losses are included    
    temp_btc_alloc_pct = temp_btc_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_eth_alloc_pct = temp_eth_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_ust3_alloc_pct = temp_ust3_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
        
    # Update the balances according to the price changes of the day:
    df_hodl.at[index + 1, 'btc_alloc_usd'] = temp_btc_alloc_usd * (1 + next_row['btcusd_dreturn'])
    df_hodl.at[index + 1, 'eth_alloc_usd'] = temp_eth_alloc_usd * (1 + next_row['ethusd_dreturn'])
    df_hodl.at[index + 1, 'ust3_alloc_usd'] = temp_ust3_alloc_usd * (1 + next_row['ust3_dreturn'])
    
    # Calculating portfolio return    
    df_hodl.at[index + 1, 'portfolio_return'] = temp_btc_alloc_pct * next_row['btcusd_dreturn'] \
                             + temp_eth_alloc_pct * next_row['ethusd_dreturn'] \
                             + temp_ust3_alloc_pct * next_row['ust3_dreturn']
    
    df_hodl.at[index+1, 'portfolio_balance_60_40'] = df_hodl.at[index + 1, 'btc_alloc_usd'] + df_hodl.at[index + 1, 'eth_alloc_usd'] + df_hodl.at[index + 1, 'ust3_alloc_usd']
    
    # Calculating allocation % again and updating datframe, after profit/losses are included  
    df_hodl.at[index + 1, 'btc_alloc%'] = df_hodl.at[index + 1,'btc_alloc_usd'] / (df_hodl.at[index + 1,'btc_alloc_usd'] + df_hodl.at[index + 1,'eth_alloc_usd'] + df_hodl.at[index + 1,'ust3_alloc_usd'])
    df_hodl.at[index + 1, 'eth_alloc%'] = df_hodl.at[index + 1,'eth_alloc_usd'] / (df_hodl.at[index + 1,'btc_alloc_usd'] + df_hodl.at[index + 1,'eth_alloc_usd'] + df_hodl.at[index + 1,'ust3_alloc_usd'])
    df_hodl.at[index + 1, 'ust3_alloc%'] = df_hodl.at[index + 1,'ust3_alloc_usd'] / (df_hodl.at[index + 1,'btc_alloc_usd'] + df_hodl.at[index + 1,'eth_alloc_usd'] + df_hodl.at[index + 1,'ust3_alloc_usd'])
    
    # Sharpe ratio:
    temp_portfolio_volatility = df_hodl.loc[:index, 'portfolio_return'].rolling(window=10).std() * np.sqrt(10)

    # Calculate daily Sharpe ratio
    temp_daily_sharpe = (df_hodl.at[index, 'portfolio_return'] - df_hodl.at[index, 'ust3_dreturn']) / temp_portfolio_volatility.iloc[-1]

    # Assign calculated values to the DataFrame
    df_hodl.at[index, 'portfolio_volatility'] = temp_portfolio_volatility.iloc[-1]
    df_hodl.at[index, 'daily_sharpe'] = temp_daily_sharpe
        
# Formatting as datetime for plotting    
df_hodl['time'] = pd.to_datetime(df_hodl['time'])

# Exporting to csv
df_hodl.to_csv('60_40_portfolio.csv', index=False)

In [122]:
# 50% BTC and 50% ETH portfolio

df_hodl2 = pd.merge(df_portfolio, prices, on='time')

# Setting up the initial asset allocations
df_hodl2['btc_alloc_usd'] = 500000
df_hodl2['eth_alloc_usd'] = 500000
df_hodl2['ust3_alloc_usd'] = 0
# Initial allocation %
df_hodl['btc_alloc%'] = 0.5
df_hodl['eth_alloc%'] = 0.5
df_hodl['ust3_alloc%'] = 0

df_hodl2['portfolio_balance_50_50'] = df_hodl2['btc_alloc_usd'] + df_hodl2['eth_alloc_usd'] + df_hodl2['ust3_alloc_usd']

for index, row in df_hodl2.iterrows():
    if index == df_hodl2.index[-1]:
        break

    next_row = df_hodl2.loc[index + 1]

    temp_btc_alloc_usd = df_hodl2.at[index, 'btc_alloc_usd']
    temp_eth_alloc_usd = df_hodl2.at[index, 'eth_alloc_usd']
    temp_ust3_alloc_usd = df_hodl2.at[index, 'ust3_alloc_usd']   
    
    # Calculating allocation % before profit/losses are included    
    temp_btc_alloc_pct = temp_btc_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_eth_alloc_pct = temp_eth_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_ust3_alloc_pct = temp_ust3_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
        
    # Update the balances according to the price changes of the day:
    df_hodl2.at[index + 1, 'btc_alloc_usd'] = temp_btc_alloc_usd * (1 + next_row['btcusd_dreturn'])
    df_hodl2.at[index + 1, 'eth_alloc_usd'] = temp_eth_alloc_usd * (1 + next_row['ethusd_dreturn'])
    df_hodl2.at[index + 1, 'ust3_alloc_usd'] = temp_ust3_alloc_usd * (1 + next_row['ust3_dreturn'])
    
    # Calculating portfolio return    
    df_hodl2.at[index + 1, 'portfolio_return'] = temp_btc_alloc_pct * next_row['btcusd_dreturn'] \
                             + temp_eth_alloc_pct * next_row['ethusd_dreturn'] \
                             + temp_ust3_alloc_pct * next_row['ust3_dreturn']
    
    df_hodl2.at[index+1, 'portfolio_balance_50_50'] = df_hodl2.at[index + 1, 'btc_alloc_usd'] + df_hodl2.at[index + 1, 'eth_alloc_usd'] + df_hodl2.at[index + 1, 'ust3_alloc_usd']
    
    # Calculating allocation % again and updating datframe, after profit/losses are included  
    df_hodl2.at[index + 1, 'btc_alloc%'] = df_hodl2.at[index + 1,'btc_alloc_usd'] / (df_hodl2.at[index + 1,'btc_alloc_usd'] + df_hodl2.at[index + 1,'eth_alloc_usd'] + df_hodl2.at[index + 1,'ust3_alloc_usd'])
    df_hodl2.at[index + 1, 'eth_alloc%'] = df_hodl2.at[index + 1,'eth_alloc_usd'] / (df_hodl2.at[index + 1,'btc_alloc_usd'] + df_hodl2.at[index + 1,'eth_alloc_usd'] + df_hodl2.at[index + 1,'ust3_alloc_usd'])
    df_hodl2.at[index + 1, 'ust3_alloc%'] = df_hodl2.at[index + 1,'ust3_alloc_usd'] / (df_hodl2.at[index + 1,'btc_alloc_usd'] + df_hodl2.at[index + 1,'eth_alloc_usd'] + df_hodl2.at[index + 1,'ust3_alloc_usd'])
    
    # Sharpe ratio:
    
    temp_portfolio_volatility = df_hodl2.loc[:index, 'portfolio_return'].rolling(window=10).std() * np.sqrt(10)

    # Calculate daily Sharpe ratio
    temp_daily_sharpe = (df_hodl2.at[index, 'portfolio_return'] - df_hodl2.at[index, 'ust3_dreturn']) / temp_portfolio_volatility.iloc[-1]

    # Assign calculated values to the DataFrame
    df_hodl2.at[index, 'portfolio_volatility'] = temp_portfolio_volatility.iloc[-1]
    df_hodl2.at[index, 'daily_sharpe'] = temp_daily_sharpe
    
# Formatting as datetime for plotting    
df_hodl2['time'] = pd.to_datetime(df_hodl2['time'])

# Exporting to csv
df_hodl2.to_csv('50_50_portfolio.csv', index=False)

In [123]:
# 100% BTC portfolio

new_test_start_date = '2022-03-14'
new_cutoff_date = '2022-09-14'

btc_hodl = pd.merge(df_portfolio, prices, on='time')

# 100% Bitcoin allocation
btc_hodl['btc_alloc_usd'] = 1000000
btc_hodl['eth_alloc_usd'] = 0
btc_hodl['ust3_alloc_usd'] = 0
# Initial allocation %
df_hodl['btc_alloc%'] = 1
df_hodl['eth_alloc%'] = 0
df_hodl['ust3_alloc%'] = 0

btc_hodl['portfolio_balance_50_50'] = btc_hodl['btc_alloc_usd'] + btc_hodl['eth_alloc_usd'] + btc_hodl['ust3_alloc_usd']

for index, row in btc_hodl.iterrows():
    if index == btc_hodl.index[-1]:
        break

    next_row = btc_hodl.loc[index + 1]

    temp_btc_alloc_usd = btc_hodl.at[index, 'btc_alloc_usd']
    temp_eth_alloc_usd = btc_hodl.at[index, 'eth_alloc_usd']
    temp_ust3_alloc_usd = btc_hodl.at[index, 'ust3_alloc_usd']   
    
    # Calculating allocation % before profit/losses are included    
    temp_btc_alloc_pct = temp_btc_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_eth_alloc_pct = temp_eth_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_ust3_alloc_pct = temp_ust3_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
        
    # Update the balances according to the price changes of the day:
    btc_hodl.at[index + 1, 'btc_alloc_usd'] = temp_btc_alloc_usd * (1 + next_row['btcusd_dreturn'])
    btc_hodl.at[index + 1, 'eth_alloc_usd'] = temp_eth_alloc_usd * (1 + next_row['ethusd_dreturn'])
    btc_hodl.at[index + 1, 'ust3_alloc_usd'] = temp_ust3_alloc_usd * (1 + next_row['ust3_dreturn'])
    
    # Calculating portfolio return    
    btc_hodl.at[index + 1, 'portfolio_return'] = temp_btc_alloc_pct * next_row['btcusd_dreturn'] \
                             + temp_eth_alloc_pct * next_row['ethusd_dreturn'] \
                             + temp_ust3_alloc_pct * next_row['ust3_dreturn']
    
    btc_hodl.at[index+1, 'portfolio_balance_50_50'] = btc_hodl.at[index + 1, 'btc_alloc_usd'] + btc_hodl.at[index + 1, 'eth_alloc_usd'] + btc_hodl.at[index + 1, 'ust3_alloc_usd']
    
    # Calculating allocation % again and updating datframe, after profit/losses are included  
    btc_hodl.at[index + 1, 'btc_alloc%'] = btc_hodl.at[index + 1,'btc_alloc_usd'] / (btc_hodl.at[index + 1,'btc_alloc_usd'] + btc_hodl.at[index + 1,'eth_alloc_usd'] + btc_hodl.at[index + 1,'ust3_alloc_usd'])
    btc_hodl.at[index + 1, 'eth_alloc%'] = btc_hodl.at[index + 1,'eth_alloc_usd'] / (btc_hodl.at[index + 1,'btc_alloc_usd'] + btc_hodl.at[index + 1,'eth_alloc_usd'] + btc_hodl.at[index + 1,'ust3_alloc_usd'])
    btc_hodl.at[index + 1, 'ust3_alloc%'] = btc_hodl.at[index + 1,'ust3_alloc_usd'] / (btc_hodl.at[index + 1,'btc_alloc_usd'] + btc_hodl.at[index + 1,'eth_alloc_usd'] + btc_hodl.at[index + 1,'ust3_alloc_usd'])
    
    # Sharpe ratio:
    
    temp_portfolio_volatility = btc_hodl.loc[:index, 'portfolio_return'].rolling(window=10).std() * np.sqrt(10)

    # Calculate daily Sharpe ratio
    temp_daily_sharpe = (btc_hodl.at[index, 'portfolio_return'] - btc_hodl.at[index, 'ust3_dreturn']) / temp_portfolio_volatility.iloc[-1]

    # Assign calculated values to the DataFrame
    btc_hodl.at[index, 'portfolio_volatility'] = temp_portfolio_volatility.iloc[-1]
    btc_hodl.at[index, 'daily_sharpe'] = temp_daily_sharpe
    
# Formatting as datetime for plotting    
btc_hodl['time'] = pd.to_datetime(btc_hodl['time'])

btc_hodl = btc_hodl.rename(columns={'portfolio_balance_50_50': '100_btc_portfolio_balance'})

# Exporting btc portfolio benchamrk as CSV file
btc_hodl.to_csv('btc_hodl.csv', index=False)

In [124]:
# 100% ETH portfolio

eth_hodl = pd.merge(df_portfolio, prices, on='time')

# 100% Ethereum allocation
eth_hodl['eth_alloc_usd'] = 1000000
eth_hodl['btc_alloc_usd'] = 0
eth_hodl['ust3_alloc_usd'] = 0
# Initial allocation %
eth_hodl['eth_alloc%'] = 1
eth_hodl['btc_alloc%'] = 0
eth_hodl['ust3_alloc%'] = 0

eth_hodl['portfolio_balance_50_50'] = eth_hodl['eth_alloc_usd'] + eth_hodl['btc_alloc_usd'] + eth_hodl['ust3_alloc_usd']

for index, row in eth_hodl.iterrows():
    if index == eth_hodl.index[-1]:
        break

    next_row = eth_hodl.loc[index + 1]

    temp_eth_alloc_usd = eth_hodl.at[index, 'eth_alloc_usd']
    temp_btc_alloc_usd = eth_hodl.at[index, 'btc_alloc_usd']
    temp_ust3_alloc_usd = eth_hodl.at[index, 'ust3_alloc_usd']   
    
    # Calculating allocation % before profit/losses are included    
    temp_eth_alloc_pct = temp_eth_alloc_usd / (temp_eth_alloc_usd + temp_btc_alloc_usd + temp_ust3_alloc_usd)
    temp_btc_alloc_pct = temp_btc_alloc_usd / (temp_eth_alloc_usd + temp_btc_alloc_usd + temp_ust3_alloc_usd)
    temp_ust3_alloc_pct = temp_ust3_alloc_usd / (temp_eth_alloc_usd + temp_btc_alloc_usd + temp_ust3_alloc_usd)
        
    # Update the balances according to the price changes of the day:
    eth_hodl.at[index + 1, 'eth_alloc_usd'] = temp_eth_alloc_usd * (1 + next_row['ethusd_dreturn'])
    eth_hodl.at[index + 1, 'btc_alloc_usd'] = temp_btc_alloc_usd * (1 + next_row['btcusd_dreturn'])
    eth_hodl.at[index + 1, 'ust3_alloc_usd'] = temp_ust3_alloc_usd * (1 + next_row['ust3_dreturn'])
    
    # Calculating portfolio return    
    eth_hodl.at[index + 1, 'portfolio_return'] = temp_eth_alloc_pct * next_row['ethusd_dreturn'] \
                             + temp_btc_alloc_pct * next_row['btcusd_dreturn'] \
                             + temp_ust3_alloc_pct * next_row['ust3_dreturn']
    
    eth_hodl.at[index+1, 'portfolio_balance_50_50'] = eth_hodl.at[index + 1, 'eth_alloc_usd'] + eth_hodl.at[index + 1, 'btc_alloc_usd'] + eth_hodl.at[index + 1, 'ust3_alloc_usd']
    
    # Calculating allocation % again and updating datframe, after profit/loss
    eth_hodl.at[index + 1, 'btc_alloc%'] = eth_hodl.at[index + 1,'btc_alloc_usd'] / (eth_hodl.at[index + 1,'btc_alloc_usd'] + eth_hodl.at[index + 1,'eth_alloc_usd'] + eth_hodl.at[index + 1,'ust3_alloc_usd'])
    eth_hodl.at[index + 1, 'eth_alloc%'] = eth_hodl.at[index + 1,'eth_alloc_usd'] / (eth_hodl.at[index + 1,'btc_alloc_usd'] + eth_hodl.at[index + 1,'eth_alloc_usd'] + eth_hodl.at[index + 1,'ust3_alloc_usd'])
    eth_hodl.at[index + 1, 'ust3_alloc%'] = eth_hodl.at[index + 1,'ust3_alloc_usd'] / (eth_hodl.at[index + 1,'btc_alloc_usd'] + eth_hodl.at[index + 1,'eth_alloc_usd'] + eth_hodl.at[index + 1,'ust3_alloc_usd'])
    
    # Sharpe ratio:
    
    temp_portfolio_volatility = eth_hodl.loc[:index, 'portfolio_return'].rolling(window=10).std() * np.sqrt(10)

    # Calculate daily Sharpe ratio
    temp_daily_sharpe = (eth_hodl.at[index, 'portfolio_return'] - eth_hodl.at[index, 'ust3_dreturn']) / temp_portfolio_volatility.iloc[-1]

    # Assign calculated values to the DataFrame
    eth_hodl.at[index, 'portfolio_volatility'] = temp_portfolio_volatility.iloc[-1]
    eth_hodl.at[index, 'daily_sharpe'] = temp_daily_sharpe
    
# Formatting as datetime for plotting    
eth_hodl['time'] = pd.to_datetime(eth_hodl['time'])

eth_hodl = eth_hodl.rename(columns={'portfolio_balance_50_50': '100_eth_portfolio_balance'})

# Exporting btc portfolio benchamrk as CSV file
eth_hodl.to_csv('eth_hodl.csv', index=False)

In [126]:
# 100% BTC portfolio, shorter timeframe for sentiment analysis

new_test_start_date = '2022-03-14'
new_cutoff_date = '2022-09-14'

btc_hodl2 = pd.merge(df_portfolio, prices, on='time')

btc_hodl2 = btc_hodl2[(btc_hodl2['time'] >= new_test_start_date) & (btc_hodl2['time'] <= new_cutoff_date)]

# 100% Bitcoin allocation
btc_hodl2['btc_alloc_usd'] = 1000000
btc_hodl2['eth_alloc_usd'] = 0
btc_hodl2['ust3_alloc_usd'] = 0
# Initial allocation %
df_hodl2['btc_alloc%'] = 0.5
df_hodl2['eth_alloc%'] = 0.5
df_hodl2['ust3_alloc%'] = 0

btc_hodl2['portfolio_balance_50_50'] = btc_hodl2['btc_alloc_usd'] + btc_hodl2['eth_alloc_usd'] + btc_hodl2['ust3_alloc_usd']

for index, row in btc_hodl2.iterrows():
    if index == btc_hodl2.index[-1]:
        break

    next_row = btc_hodl2.loc[index + 1]

    temp_btc_alloc_usd = btc_hodl2.at[index, 'btc_alloc_usd']
    temp_eth_alloc_usd = btc_hodl2.at[index, 'eth_alloc_usd']
    temp_ust3_alloc_usd = btc_hodl2.at[index, 'ust3_alloc_usd']   
    
    # Calculating allocation % before profit/losses are included    
    temp_btc_alloc_pct = temp_btc_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_eth_alloc_pct = temp_eth_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
    temp_ust3_alloc_pct = temp_ust3_alloc_usd / (temp_btc_alloc_usd + temp_eth_alloc_usd + temp_ust3_alloc_usd)
        
    # Update the balances according to the price changes of the day:
    btc_hodl2.at[index + 1, 'btc_alloc_usd'] = temp_btc_alloc_usd * (1 + next_row['btcusd_dreturn'])
    btc_hodl2.at[index + 1, 'eth_alloc_usd'] = temp_eth_alloc_usd * (1 + next_row['ethusd_dreturn'])
    btc_hodl2.at[index + 1, 'ust3_alloc_usd'] = temp_ust3_alloc_usd * (1 + next_row['ust3_dreturn'])
    
    # Calculating portfolio return    
    btc_hodl2.at[index + 1, 'portfolio_return'] = temp_btc_alloc_pct * next_row['btcusd_dreturn'] \
                             + temp_eth_alloc_pct * next_row['ethusd_dreturn'] \
                             + temp_ust3_alloc_pct * next_row['ust3_dreturn']
    
    btc_hodl2.at[index+1, 'portfolio_balance_50_50'] = btc_hodl2.at[index + 1, 'btc_alloc_usd'] + btc_hodl2.at[index + 1, 'eth_alloc_usd'] + btc_hodl2.at[index + 1, 'ust3_alloc_usd']
    
    # Calculating allocation % again and updating datframe, after profit/losses are included  
    btc_hodl2.at[index + 1, 'btc_alloc%'] = btc_hodl2.at[index + 1,'btc_alloc_usd'] / (btc_hodl2.at[index + 1,'btc_alloc_usd'] + btc_hodl2.at[index + 1,'eth_alloc_usd'] + btc_hodl2.at[index + 1,'ust3_alloc_usd'])
    btc_hodl2.at[index + 1, 'eth_alloc%'] = btc_hodl2.at[index + 1,'eth_alloc_usd'] / (btc_hodl2.at[index + 1,'btc_alloc_usd'] + btc_hodl2.at[index + 1,'eth_alloc_usd'] + btc_hodl2.at[index + 1,'ust3_alloc_usd'])
    btc_hodl2.at[index + 1, 'ust3_alloc%'] = btc_hodl2.at[index + 1,'ust3_alloc_usd'] / (btc_hodl2.at[index + 1,'btc_alloc_usd'] + btc_hodl2.at[index + 1,'eth_alloc_usd'] + btc_hodl2.at[index + 1,'ust3_alloc_usd'])
    
    # Sharpe ratio:
    
    temp_portfolio_volatility = btc_hodl2.loc[:index, 'portfolio_return'].rolling(window=10).std() * np.sqrt(10)

    # Calculate daily Sharpe ratio
    temp_daily_sharpe = (btc_hodl2.at[index, 'portfolio_return'] - btc_hodl2.at[index, 'ust3_dreturn']) / temp_portfolio_volatility.iloc[-1]

    # Assign calculated values to the DataFrame
    btc_hodl2.at[index, 'portfolio_volatility'] = temp_portfolio_volatility.iloc[-1]
    btc_hodl2.at[index, 'daily_sharpe'] = temp_daily_sharpe
    
# Formatting as datetime for plotting    
btc_hodl2['time'] = pd.to_datetime(btc_hodl['time'])

btc_hodl2 = btc_hodl2.rename(columns={'portfolio_balance_50_50': '100_btc_portfolio_balance'})

# Exporting btc portfolio benchamrk as CSV file
btc_hodl2.to_csv('btc_hodl2.csv', index=False)